In [1]:
import warnings
warnings.simplefilter("ignore")
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt
%matplotlib inline
import gensim
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from keras.initializers import Constant
from keras.layers import (Input, Embedding, SpatialDropout1D, Bidirectional,
                          GlobalAveragePooling1D, GlobalMaxPooling1D,
                          LSTM, CuDNNLSTM, GRU, CuDNNGRU, concatenate, Dropout, Dense, Activation)
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
NUM_WORDS = 50000
MAXLEN = 250
EMBEDDING_DIM = 300

In [3]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data):
        self.X_val, self.y_val = validation_data
    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.X_val, verbose=0)
        score = roc_auc_score(self.y_val, y_pred)
        print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch + 1, score))

In [4]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
X_train_raw, y_train = train['comment_text'].str.lower(), train[classes].values
X_test_raw = test['comment_text'].str.lower()
tk = Tokenizer(num_words=NUM_WORDS)
tk.fit_on_texts(X_train_raw)
X_train = tk.texts_to_sequences(X_train_raw)
X_test = tk.texts_to_sequences(X_test_raw)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=0)
X_train = pad_sequences(X_train, maxlen=MAXLEN)
X_valid = pad_sequences(X_valid, maxlen=MAXLEN)
X_test = pad_sequences(X_test, maxlen=MAXLEN)
nb_words = min(NUM_WORDS, len(tk.word_index) + 1)

In [5]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def get_embedding(pretrained_word_vectors):
    if pretrained_word_vectors == "google":
        EMBEDDING_PATH = "../embeddings/GoogleNews-vectors-negative300.bin"
        embedding_index = gensim.models.KeyedVectors.load_word2vec_format(EMBEDDING_PATH, binary=True)
    if pretrained_word_vectors == "glove":
        EMBEDDING_PATH = "../embeddings/glove.840B.300d.txt"
        embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(EMBEDDING_PATH))
    elif pretrained_word_vectors == "fasttext":
        EMBEDDING_PATH = "../embeddings/crawl-300d-2M.vec"
        embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(EMBEDDING_PATH))
    embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
    for word, i in tk.word_index.items():
        if i >= NUM_WORDS:
            continue
        if pretrained_word_vectors == "google":
            try:
                embedding_vector = embedding_index.get_vector(word)
            except Exception:
                continue
            embedding_matrix[i] = embedding_vector
        else:
            embedding_vector = embedding_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [6]:
def build_model(embedding_matrix):
    inputs = Input(shape=(MAXLEN,))
    x = Embedding(nb_words, EMBEDDING_DIM,
                  embeddings_initializer=Constant(embedding_matrix),
                  trainable=False)(inputs)
    x = SpatialDropout1D(0.2)(x)
    x1 = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    x2 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    avg_pool1 = GlobalAveragePooling1D()(x1)
    max_pool1 = GlobalMaxPooling1D()(x1)
    avg_pool2 = GlobalAveragePooling1D()(x2)
    max_pool2 = GlobalMaxPooling1D()(x2)
    x = concatenate([avg_pool1, max_pool1, avg_pool2, max_pool2])
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
    return model

In [7]:
embedding_matrix = get_embedding("glove")
roc_eval = RocAucEvaluation(validation_data=(X_valid, y_valid))
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
model = build_model(embedding_matrix=embedding_matrix)
model.fit(X_train, y_train, batch_size=128, epochs=10,
          validation_data=(X_valid, y_valid),
          callbacks = [roc_eval, early_stopping])
pred = model.predict(X_test, batch_size=1024, verbose=1)
submission = pd.read_csv("data/sample_submission.csv")
submission[classes] = pred
submission.to_csv("submission/v4_BiGRU_BiLSTM_glove_submission.csv.gz", compression="gzip", index=False)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 143613 samples, validate on 15958 samples
Epoch 1/10
143613/143613 [==============================] - 84s 585us/step - loss: 0.0596 - acc: 0.9792 - val_loss: 0.0433 - val_acc: 0.9835

 ROC-AUC - epoch: 1 - score: 0.984583
Epoch 2/10
143613/143613 [==============================] - 84s 588us/step - loss: 0.0446 - acc: 0.9830 - val_loss: 0.0408 - val_acc: 0.9841

 ROC-AUC - epoch: 2 - score: 0.989320
Epoch 3/10
143613/143613 [==============================] - 85s 594us/step - loss: 0.0414 - acc: 0.9837 - val_loss: 0.0396 - val_acc: 0.9844

 ROC-AUC - epoch: 3 - score: 0.989800
Epoch 4/10
143613/143613 [==============================] - 85s 595us/step - loss: 0.0393 - acc:

In [8]:
embedding_matrix = get_embedding("fasttext")
roc_eval = RocAucEvaluation(validation_data=(X_valid, y_valid))
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
model = build_model(embedding_matrix=embedding_matrix)
model.fit(X_train, y_train, batch_size=128, epochs=10,
          validation_data=(X_valid, y_valid),
          callbacks = [roc_eval, early_stopping])
pred = model.predict(X_test, batch_size=1024, verbose=1)
submission = pd.read_csv("data/sample_submission.csv")
submission[classes] = pred
submission.to_csv("submission/v4_BiGRU_BiLSTM_fasttext_submission.csv.gz", compression="gzip", index=False)

Train on 143613 samples, validate on 15958 samples
Epoch 1/10
143613/143613 [==============================] - 82s 571us/step - loss: 0.0618 - acc: 0.9786 - val_loss: 0.0457 - val_acc: 0.9822

 ROC-AUC - epoch: 1 - score: 0.984234
Epoch 2/10
143613/143613 [==============================] - 86s 597us/step - loss: 0.0452 - acc: 0.9828 - val_loss: 0.0405 - val_acc: 0.9845

 ROC-AUC - epoch: 2 - score: 0.987443
Epoch 3/10
143613/143613 [==============================] - 86s 596us/step - loss: 0.0423 - acc: 0.9837 - val_loss: 0.0396 - val_acc: 0.9843

 ROC-AUC - epoch: 3 - score: 0.989650
Epoch 4/10
143613/143613 [==============================] - 87s 604us/step - loss: 0.0400 - acc: 0.9843 - val_loss: 0.0398 - val_acc: 0.9841

 ROC-AUC - epoch: 4 - score: 0.989663
Epoch 5/10
143613/143613 [==============================] - 86s 597us/step - loss: 0.0382 - acc: 0.9849 - val_loss: 0.0394 - val_acc: 0.9843

 ROC-AUC - epoch: 5 - score: 0.989819
Epoch 6/10
143613/143613 [=======================